In [1]:
from collections import defaultdict
from pathlib import Path

import json

from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch

In [6]:
def get_headers(df):
    keys = list(df.keys())[1:]
    headers = list()
    for key in keys:
        key = '_'.join(key.split('_')[:-2]).lower()
        if key not in headers:
            headers.append(key)
        else:
            break
    assert(len(set(headers)) == len(headers))
    return headers

In [7]:
data = defaultdict(dict)
avg_in_KG = 0
avg_out_KG = 0
avg_in_rank = 0
avg_out_rank = 0
avg_in_MC = 0
avg_out_MC = 0
avg_train_acc = 0
avg_test_acc = 0
avg_qc = 0
empty = True
avg_train_loss = 0
avg_test_loss = 0
f=Path(str(sys.path[0][0:-18])+"/outputs/results_date2021-05-14-13-41-09_trial0_ResNet34CIFAR_CIFAR10_Adasmomentum0.9_weight_decay0.0005_beta.xlsx")
if f.suffix == '.xlsx':
    df = pd.read_excel(f)
    headers = get_headers(df)
    print(headers)
    print(f.name)
    df = df.T
    in_KG = np.asarray(
        df.iloc[headers.index('in_s') + 1::len(headers),
                :])
    #assert(len(in_KG[:, -1]) == 250)
    out_KG = np.asarray(
        df.iloc[headers.index('out_s') + 1::len(headers),
                :])
    in_rank = np.asarray(
        df.iloc[headers.index('in_rank') + 1::len(headers),
                :])
    out_rank = np.asarray(
        df.iloc[headers.index('out_rank') + 1::len(headers),
                :])
    in_MC = np.asarray(
        df.iloc[headers.index('in_condition') + 1::len(headers),
                :])
    out_MC = np.asarray(
        df.iloc[headers.index('out_condition') + 1::len(headers),
                :])
    in_MC[in_MC == 0] = 500
    out_MC[out_MC == 0] = 500
    avg_in_KG += in_KG
    avg_out_KG += out_KG
    avg_in_rank += in_rank
    avg_out_rank += out_rank
    avg_in_MC += in_MC
    avg_out_MC += out_MC
    QC = np.arctan2((in_KG[-1, :] + out_KG[-1, :]), 
        1 - 1 / ((in_MC[-1, :]  + out_MC[-1, :])))
    QC = np.linalg.norm(QC) / np.sqrt(in_KG.shape[1])
    avg_qc += QC
    tag = 'test_acc1'
    if 'test_acc1' not in headers:
        tag = 'test_acc'
        if 'test_acc' not in headers:
            tag = 'acc'
    test_acc = np.asarray(
        df.iloc[headers.index(tag) + 1::len(headers), :])
    tag = 'train_acc1'
    avg_test_acc += test_acc
    if 'train_acc1' not in headers:
        tag = 'train_acc'
    try:
        train_acc = np.asarray(
            df.iloc[headers.index(tag) + 1::len(headers), :])
        avg_train_acc += train_acc
    except Exception:
        avg_train_acc += np.zeros_like(avg_train_acc)
    train_loss = np.asarray(
                df.iloc[headers.index('train_loss') + 1::len(headers), :])
    avg_train_loss += train_loss
    try:
        test_loss = np.asarray(
                    df.iloc[headers.index('test_loss') + 1::len(headers), :])
        avg_test_loss += test_loss
    except Exception:
        avg_test_loss += np.zeros_like(avg_test_loss)
data['wasda'] = defaultdict()
data['wasda']['asdfadsf'] = defaultdict()
data['wasda']['asdfadsf']['dasfdfs'] = {
    "in_KG": avg_in_KG,
    "out_KG": avg_out_KG,
    "in_rank": avg_in_rank,
    "out_rank": avg_out_rank,
    "in_MC": avg_in_MC,
    "out_MC": avg_out_MC,
    "QC": avg_qc,
    "train_loss": avg_train_loss,
    "test_loss": avg_test_loss,
    "train_acc": avg_train_acc,
    "test_acc": avg_test_acc,
}
                # n_grid = np.round(np.linspace(
                #     0, QC.shape[0]-1, QC.shape[1]))
                # n_grid = n_grid.astype(int)
                # plt.imshow(QC[n_grid, :].T,
                #            cmap='jet', vmin=0, vmax=np.pi/4)
                # locs, labels = plt.xticks()
                # x_labels = np.round(np.linspace(1, len(n_grid), len(locs)))
                # x_labels = x_labels.astype(int)
                # x_labels = n_grid[x_labels-1]
                # plt.xticks(locs, x_labels)
                # fname = f'figures/{data_dir.name}_{exp_dir.name}_{opt_dir.name}.png'
                

['train_acc1', 'train_acc5', 'train_loss', 'in_s', 'out_s', 'fc_s', 'in_rank', 'out_rank', 'fc_rank', 'in_condition', 'out_condition', 'rank_velocity', 'learning_rate', 'test_acc1', 'test_acc5', 'test_loss']
results_date2021-05-14-13-41-09_trial0_ResNet34CIFAR_CIFAR10_Adasmomentum0.9_weight_decay0.0005_beta.xlsx


In [11]:
qc = list()
for key, metrics in data['wasda']['asdfadsf'].items():
    QC = np.arctan2((metrics['in_KG'][-1, :] + metrics['out_KG'][-1, :]), 
                            1 - 1 / ((metrics['in_MC'][-1, :]  + metrics['out_MC'][-1, :])))
    print(str(metrics['in_KG'][-1, :]),str(metrics['out_KG'][-1, :]),str(metrics['in_MC'][-1, :]),str(metrics['out_MC'][-1, :]),str(QC))
    try:
        QC = np.linalg.norm(QC) / np.sqrt(metrics['in_KG'].shape[1])
        if not np.isnan(QC):
            qc.append(QC)
    except:
        print("0")
qc = np.array(qc)
print(qc)
        

[0.52376898 0.13447009 0.13970384 0.13560082 0.16551089 0.16026661
 0.13521574 0.18935299 0.14689079 0.12855875 0.11127771 0.1868922
 0.12495261 0.13445675 0.12939091 0.11487824 0.177935   0.12807907
 0.12917294 0.11652815 0.13405395 0.10917497 0.12420331 0.12322948
 0.10397523 0.12461573 0.09172168 0.12390505 0.07692295 0.16142043
 0.07666598 0.09028235 0.04128625 0.04654006 0.04385948 0.03285949] [0.10588269 0.15915146 0.2127592  0.17454192 0.18552665 0.15033413
 0.1581198  0.14864792 0.20742509 0.06427238 0.16918537 0.18574788
 0.15628409 0.13981055 0.13846271 0.12162465 0.13157451 0.16803366
 0.06579854 0.14810064 0.12933049 0.12148054 0.12700517 0.12391229
 0.10339642 0.11202975 0.09583086 0.1177469  0.07066889 0.09347329
 0.07133174 0.04543473 0.05510183 0.04646784 0.04791275 0.02898097] [ 1.75037265 72.21595764 18.27045441 56.94750595 21.39185905 11.99610043
 29.24967766 15.67850304 17.87739372  5.09984684 21.23066521 15.06327343
 18.76285744 26.30274391 18.03287697 28.78133774 

In [7]:
torch.save(data, 'data/data_full.pt')